<a href="https://colab.research.google.com/github/okadatsuneo/Plateau_import/blob/main/uploadosm%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Plateauインポートのuploadファイルの処理：既存wayのノードが削除されるので、uploadファイルにある既存wayの既存ノードをorgファイルから持ってくる。
変更後のuploadファイル(ここの出力ファイル:upload_2.osm)をJOSMで開き、単独ノードを近くにあるwayのノードに手動で統合する。

入力ファイル：uploadとop.orgの2つのOSMファイル

出力ファイル：upload_2.osm

In [2]:
import os
import xml.etree.ElementTree as ET
import copy
from google.colab import files


# ファイルをアップロード
print("uploadのOSMファイルを選択してください。")
uploaded_files = files.upload()
file_path_upload = list(uploaded_files.keys())[0]

print("op.orgのOSMファイルを選択してください。")
uploaded_files = files.upload()
file_path_org = list(uploaded_files.keys())[0]

# 出力ファイルのパスを決定
file_name, file_ext = os.path.splitext(file_path_upload)
file_path_output = f"{file_name}_2{file_ext}"

# XMLファイルをパースする
tree_upload = ET.parse(file_path_upload)
tree_org = ET.parse(file_path_org)

# ルート要素を取得
root_upload = tree_upload.getroot()
root_org = tree_org.getroot()

# 処理したいwayidのセット(action='modify'かつidが正の数)
subject_ways = set()

# uploadファイルのway要素から処理したいものを辞書に格納:subject_ways
for way in root_upload.findall('.//way'):
    action = way.get("action")
    id_attr = way.get("id")

    # 'action'が 'modify' で 'id'が正の数の場合の処理
    if action == 'modify' and id_attr.isdigit() and int(id_attr) > 0:
        subject_ways.add(id_attr)

# 出力ファイル用のXMLツリーをuploadからコピー
output_root = copy.deepcopy(root_upload)
output_tree = ET.ElementTree(output_root)

# subject_waysのidのwayに対して処理をする。
for way_id in subject_ways:
    # root_orgのway要素を取得
    org_way = root_org.find(f".//way[@id='{way_id}']")

    if org_way is not None:
        # way要素内のnd要素を操作
        for nd in org_way.findall("nd"):
            ref = nd.get("ref")

            # ref属性と同じidを持つnodeをroot_orgから探す
            org_node = root_org.find(f".//node[@id='{ref}']")

            if org_node is not None:
                org_node_id = org_node.get('id')

                # output_tree内で同じidを持つノードを探す
                output_node = output_tree.find(f'.//node[@id="{org_node_id}"]')

                if output_node is not None:
                    # 同じidを持つノードが存在する場合、action属性を変更
                    output_node.set('action', 'modify')
                else:
                    # 同じidを持つノードが存在しない場合、org_nodeをoutput_treeに追加
                    output_tree.getroot().append(org_node)

# 出力ファイルに書き込む
output_tree.write(file_path_output, encoding="utf-8", xml_declaration=True)

# ファイルをダウンロード
files.download(file_path_output)

print("処理が完了しました。",file_path_output,"をダウンロードしました。")

uploadのOSMファイルを選択してください。


Saving 51335275_upload.osm to 51335275_upload.osm
op.orgのOSMファイルを選択してください。


Saving 51335275_bldg_6697_op.org.osm to 51335275_bldg_6697_op.org.osm


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

処理が完了しました。 51335275_upload_2.osm をダウンロードしました。
